In [152]:
import numpy as np

In [153]:
xdim = 4
cons_dim = 2

# Create a symmetric positive definite matrix A
A = np.random.randn(cons_dim,xdim)
# A = A.T @ A
sample_x = np.random.randint(1,10,xdim)
b = A@sample_x
print(sample_x)
# b = np.random.randn(cons_dim)
u = np.ones(xdim)
v = np.ones(cons_dim)
# c = A.T @ np.random.randn(cons_dim) + np.random.randn(xdim)
c = np.ones(xdim)

# Use scipy to solve the optimization problem
from scipy.optimize import linprog
res = linprog(c, A_eq=A, b_eq=b, bounds=(0, None))
print(res)

[5 3 1 4]
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 8.458587227413856
              x: [ 5.665e+00  0.000e+00  0.000e+00  2.794e+00]
            nit: 0
          lower:  residual: [ 5.665e+00  0.000e+00  0.000e+00  2.794e+00]
                 marginals: [ 0.000e+00  1.082e+00  1.297e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [ 6.520e-01 -8.546e-01]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


In [154]:
def LHS(x, u, A=A):
    return np.block([
        [np.zeros((xdim, xdim )), np.eye(xdim), A.T],
        [np.diag(u), np.diag(x), np.zeros((xdim, cons_dim))],
        [A, np.zeros((cons_dim, xdim)), np.zeros((cons_dim, cons_dim))]
        ]).astype(np.float64)

def RHS(x, u, v, t, c=c, A=A, b=b):
    return -1 * np.block([A.T@v+u-c, np.diag(x)@u-t*np.ones(xdim), A@x-b]).astype(np.float64)

def residuals(x, u, v, t, c=c, A=A, b=b):
    res = RHS(x, u, v, t, c, A, b)
    r_dual = res[:xdim]
    r_prime = res[xdim+xdim:]
    return np.linalg.norm(r_dual)**2 + np.linalg.norm(r_prime)**2

In [155]:
def solve_direction(x, u, v, t, c=c, A=A, b=b):
    delta = np.linalg.solve(LHS(x, u, A), RHS(x, u, v, t, c, A, b))
    # x, u, v
    return delta[:xdim], delta[xdim:xdim+xdim], delta[xdim+xdim:]

In [226]:
# set the seed
# np.random.seed()
x = np.array([1,1,1,1])
mu=10
t = 1
alpha, beta = 0.1, 0.8
iter_num = 0
while True:
    iter_num += 1
    # solve direction
    delta_x, delta_u, delta_v = solve_direction(x, u, v, t)
    if np.all(delta_u >= 0):
        s_max = 1
    else:
        test = np.min(-u[delta_u<0]/delta_u[delta_u<0])
        s_max = np.min([1, test])
    s=0.99*s_max
    right = np.linalg.norm(RHS(x, u, v, t))
    while True:
        if np.all(x + s*delta_x >= 0):
            left = np.linalg.norm(RHS(x+s*delta_x, u+s*delta_u, v+s*delta_v,t))
            if left <= (1-alpha*s)*right:
                break
            break
        else:
            # print(x + s*delta_x)
            s = beta*s
    print(s)
    x = x + s*delta_x
    u = u + s*delta_u
    v = v + s*delta_v
    print(x)
    # print(residuals(x, u, v, t))
    if residuals(x, u, v, t) <= 1e-6:
        break
    t = mu*t
print(x)

0.17775766416462396
[1.5870428  1.52783615 1.47514535 1.81224234]
0.313648947159937
[2.41758151 2.32530272 2.24389387 3.03795435]
0.5255857903216796
[3.37228293 3.24347549 3.12987099 4.44882352]
0.7696701548031651
[4.03553321 3.88138709 3.74543638 5.42903211]
0.9392513443563149
[4.2219582  4.0606908  3.91845954 5.70454819]
0.9869170864710494
[4.233858   4.07213604 3.92950389 5.72213482]
0.9899596665378659
[4.23401416 4.07228624 3.92964883 5.72236561]
0.9899995949704874
[4.23401573 4.07228775 3.92965028 5.72236793]
0.9899999960191155
[4.23401574 4.07228777 3.9296503  5.72236796]
0.9899999999182338
[4.23401574 4.07228777 3.9296503  5.72236796]
0.99
[4.23401574 4.07228777 3.9296503  5.72236796]
0.9899999999617523
[4.23401575 4.07228777 3.9296503  5.72236796]
0.99
[4.23401574 4.07228777 3.9296503  5.72236796]
0.99
[4.23401574 4.07228777 3.9296503  5.72236796]
0.9899999999371725
[4.23401575 4.07228777 3.9296503  5.72236796]
0.99
[4.23401575 4.07228777 3.9296503  5.72236796]
0.99
[4.23401574

OverflowError: int too large to convert to float

In [157]:
residuals(x, u, v, t)

2.2017085281988984e-07